# **Chapter 5.  쇼핑몰 데이터베이스 만들기**


---
### 📝 **학습 목차**
> 1. csv 파일 다루기 - csv <br>
> 2. 데이터 베이스 연결 - sqlight3 <br>
> **3. sqlight 다루기** <br>
> 4. 데이터 백업하기 - iterdump <br>

## 3. sqlight 다루기

> ### sqlite3 데이터 조회
> - sqlite3 에서 데이터 조회 방법에는 `fetchone()`, `fetchmany()`, `fetchall()` 3 가지 방법을 사용
> - `SELECT` 문을 사용한 조회 결과 범위에서 실제 가져오는 row 수를 결정

| 메서드 | 내용 |
|-----------|-----------|
| fetchone()        | 조회 결과에서 1개의 row 를 가져옴   |
| fetchmany(size=2)   | 조회 결과에서 지정한 size 만큼 row 를 가져옴     |
| fetchall()   | 조회 결과에서 모든 row 를 가져옴      |

### DB 연결

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
# 연습용 DB 연결
conn = sqlite3.connect("chadwick.db")

In [ ]:
# 커서 생성
c = conn.cursor()

In [ ]:
# 전체 테이블 현황 조회
c.execute('''SELECT * FROM sqlite_master WHERE type="table"''')

In [ ]:
pd.DataFrame(c.fetchall())

### 조회

#### 데이터 조회

In [ ]:
# 전체 데이터 조회
c.execute('''SELECT * FROM Parks''')

In [ ]:
# # 전체 로우(ROW) 선택
# pd.DataFrame(c.fetchall())

In [ ]:
# 1개 로우 선택
pd.DataFrame(c.fetchone())

In [ ]:
# 지정 로우 선택
pd.DataFrame(c.fetchmany(size=3))

#### 특정 데이터 조회

In [ ]:
# 컬럼(키) 확인
cur = c.execute('''SELECT * FROM Parks''')
cur.description

In [ ]:
# 특정 row 만 가져오기
c.execute('''SELECT * FROM Parks WHERE state = ?''', ('NY',))
pd.DataFrame(c.fetchall())

In [ ]:
# DB 연결 해제
conn.close()

### 기본 문법

> - DB 쿼리문 작성 명령어 그대로 사용

#### 연결(pre-do)

In [2]:
import sqlite3

In [3]:
# 연습용 DB 생성 / 연결
conn = sqlite3.connect("test.db")

In [4]:
# 커서 생성
c = conn.cursor()

#### 구문

>  cursor.execute(sql_query, [parameters]) <br>
> - sql_query: 실행할 SQL 쿼리문. 문자열(주로 ''' ''') 안에 작성. <br>문법은 쿼리 문법과 동일 <br>
> - parameters (선택사항): SQL 쿼리에 전달할 매개변수. <br>
이는 튜플(tuple) 또는 딕셔너리(dictionary) 형태로 제공되며, 매개변수를 사용하여 동적으로 생성하고 실행

예시(조회)<br>
- 파라미터 없이 쿼리 실행 <br>
cursor.execute('SELECT * FROM table_name') <br>
<br>
- 파라미터를 사용하여 쿼리 실행 (플레이스홀더를 사용하는 경우)<br>
cursor.execute('SELECT * FROM table_name WHERE column_name = ?', ('value',))<br>
<br>
- 딕셔너리 형태로 파라미터를 전달하는 경우<br>
cursor.execute('SELECT * FROM table_name WHERE column_name = :value', {'value': 'some_value'})


#### 커밋(commit) 및 롤백(rollback)

conn.commit()  # 변경사항 저장 <br>
conn.rollback()  # 변경사항 취소

#### CREATE

```python
c.execute('''CREATE TABLE TableName (
                column1 datatype PRIMARY KEY,
                column2 datatype,
                ...
            )''')

```


#### INSERT

```python
c.execute('''INSERT INTO TableName (column1, column2, ...) 
              VALUES (value1, value2, ...)''')

```


##### 열(키) 항목 순서를 정확히 모르는 경우

```python
c.execute("INSERT INTO test(PRODUCT_NAME, PRICE, ID) VALUES(?,?,?)", ('티셔츠', 20000, 3))
```


##### 여러 데이터를 한번에 삽입하고 싶은 경우

```python
# 추가할 상품 리스트
product_list = [[1, '모자', 15000],
                [2, '코트', 200000],
                [3, '티셔츠', 20000],
                [4, '블라우스', 55000],
                [5, '가디건', 45000],
                [6, '청바지', 50000],
                [7, '구두', 150000],
                [8, '가방', 170000]]
```


```python
c.executemany("INSERT INTO test(ID, PRODUCT_NAME, PRICE) VALUES(?,?,?)", product_list)
```


#### UPDATE

```python
c.execute('''UPDATE TableName SET column1 = value1 WHERE condition''')
```


##### 튜플 형태로 수정

```python
c.execute("UPDATE test SET PRODUCT_NAME = ? WHERE ID = ?", ('슬랙스', 6))
```


##### 딕셔너리 형태로 수정

```python
c.execute("UPDATE test SET PRICE = :price WHERE ID = :id", {"price":55000, "id":6})
```

##### %s 표시자 사용

```python
c.execute("UPDATE test SET PRODUCT_NAME = '%s' WHERE ID = '%s'" % ('트랜치코트', 2))
```

#### DELETE

```python
c.execute('''DELETE FROM TableName WHERE condition''')
```

##### 튜플 형태로 삭제

```python
c.execute("DELETE FROM test WHERE ID =?", (8,))
```

##### 딕셔너리 형태로 삭제

```python
c.execute("DELETE FROM test WHERE PRODUCT_NAME = :product_name", {'product_name':'슬랙스'})
```

##### 전체 삭제

```python
c.execute("DELETE FROM test")
```